This notebook demonstrates the capabilities of the Chronicle Security Operations SDK. We'll explore:
* Authentication with Google Cloud
* Querying security events
* Analyzing alerts and cases
* Visualizing security data
* Working with IoCs

## Setup
First, let's install the required packages and authenticate.

In [ ]:
!pip uninstall secops -y
!pip install secops
!pip install pandas plotly

from google.colab import auth
from google.auth import default
import google.auth.transport.requests

# Authenticate
auth.authenticate_user()
creds, _ = google.auth.default()
auth_req = google.auth.transport.requests.Request()
creds.refresh(auth_req)

print("✅ Authentication successful!")

## Initialize the Chronicle Client

Now that we're authenticated, let's set up our Chronicle client. We'll use a demo tenant for this example.

In [ ]:
from datetime import datetime, timedelta, timezone
from secops import SecOpsClient
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Initialize client
client = SecOpsClient()
chronicle = client.chronicle(
    customer_id="c3c6260c1c9340dcbbb802603bbfffff",  # not a real customer id
    project_id="725716779999",
    region="us",
)

# Set time range for queries
end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(hours=24)

print("📊 Analyzing security data from:")
print(
    f"   {start_time.strftime('%Y-%m-%d %H:%M:%S')} to {end_time.strftime('%Y-%m-%d %H:%M:%S')} UTC"
)

## 🚨 Alert Analysis

Let's start by analyzing recent alerts. We'll create visualizations to understand:
* Alert severity distribution
* Top alert rules
* Alert status breakdown
* Timeline of alert creation

In [ ]:
# Get alerts
print("Fetching alerts...")
alerts = chronicle.get_alerts(start_time=start_time, end_time=end_time, max_alerts=1000)

alert_list = alerts.get("alerts", {}).get("alerts", [])
print(f"Found {len(alert_list)} alerts")

# Convert to DataFrame for analysis
alert_data = []
for alert in alert_list:
    alert_data.append(
        {
            "id": alert.get("id"),
            "rule_name": alert.get("detection", [{}])[0].get("ruleName"),
            "severity": alert.get("feedbackSummary", {}).get(
                "severityDisplay", "Unknown"
            ),
            "status": alert.get("feedbackSummary", {}).get("status", "Unknown"),
            "created_time": pd.to_datetime(alert.get("createdTime")),
            "risk_score": alert.get("feedbackSummary", {}).get("riskScore", 0),
        }
    )

df_alerts = pd.DataFrame(alert_data)

# Create visualizations
fig1 = px.pie(df_alerts, names="severity", title="Alert Severity Distribution")
fig1.show()

fig2 = px.bar(
    df_alerts["rule_name"].value_counts().head(10),
    title="Top 10 Alert Rules",
    labels={"index": "Rule Name", "value": "Count"},
)
fig2.show()

fig3 = px.histogram(
    df_alerts,
    x="created_time",
    title="Alert Timeline",
    labels={"created_time": "Creation Time", "count": "Number of Alerts"},
)
fig3.show()

## 📁 Case Analysis

Now let's examine the cases associated with these alerts. We'll create an interactive dashboard showing:
* Case priority levels
* Case stages
* Alerts per case

In [ ]:
# Get cases from alerts
case_ids = {alert.get("caseName") for alert in alert_list if alert.get("caseName")}

if case_ids:
    print(f"Analyzing {len(case_ids)} cases...")
    cases = chronicle.get_cases(list(case_ids))

    # Prepare case data
    case_data = []
    for case in cases.cases:
        case_alerts = [a for a in alert_list if a.get("caseName") == case.id]
        high_sev_alerts = [
            a
            for a in case_alerts
            if a.get("feedbackSummary", {}).get("severityDisplay") == "HIGH"
        ]

        case_data.append(
            {
                "id": case.id,
                "name": case.display_name,
                "priority": case.priority,
                "stage": case.stage,
                "status": case.status,
                "alert_count": len(case_alerts),
                "high_sev_count": len(high_sev_alerts),
            }
        )

    df_cases = pd.DataFrame(case_data)

    # Create case dashboard
    fig = go.Figure()

    # Priority distribution
    fig.add_trace(
        go.Bar(
            x=df_cases["priority"].value_counts().index,
            y=df_cases["priority"].value_counts().values,
            name="Priority Distribution",
        )
    )

    # Add alerts per case
    fig.add_trace(
        go.Scatter(
            x=df_cases["name"],
            y=df_cases["alert_count"],
            mode="markers",
            name="Alerts per Case",
            marker=dict(size=df_cases["high_sev_count"] * 5 + 10),
        )
    )

    fig.update_layout(
        title="Case Analysis Dashboard",
        xaxis_title="Cases",
        yaxis_title="Count",
        showlegend=True,
    )

    fig.show()

    # Print summary statistics
    print("\n📊 Case Summary:")
    print(f"Total Cases: {len(df_cases)}")
    print(
        f"High Priority Cases: {len(df_cases[df_cases['priority'] == 'PRIORITY_HIGH'])}"
    )
    print(f"Average Alerts per Case: {df_cases['alert_count'].mean():.1f}")
else:
    print("No cases found in the current time range")

## 🔍 Entity Investigation

Let's investigate some entities from our alerts. We'll create a network graph of related entities and their relationships.

In [ ]:
import networkx as nx

# Extract entities from alerts
entities = set()
relationships = []

for alert in alert_list:
    # Get IPs and hostnames from alert events
    for elem in alert.get("collectionElements", []):
        for ref in elem.get("references", []):
            event = ref.get("event", {})

            # Get principal IPs
            principal_ips = event.get("principal", {}).get("ip", [])
            for ip in principal_ips:
                entities.add(("ip", ip))

            # Get target IPs
            target_ips = event.get("target", {}).get("ip", [])
            for tip in target_ips:
                entities.add(("ip", tip))

                # Create relationship between source and target
                for sip in principal_ips:
                    relationships.append((sip, tip))

# Create network graph
G = nx.Graph()

# Add nodes and edges
for entity_type, entity_value in entities:
    G.add_node(entity_value, type=entity_type)

for source, target in relationships:
    G.add_edge(source, target)

# Create interactive network visualization
pos = nx.spring_layout(G)

edge_trace = go.Scatter(
    x=[], y=[], line=dict(width=0.5, color="#888"), hoverinfo="none", mode="lines"
)

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace["x"] += (x0, x1, None)
    edge_trace["y"] += (y0, y1, None)

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode="markers+text",
    hoverinfo="text",
    marker=dict(size=10, line_width=2),
)

for node in G.nodes():
    x, y = pos[node]
    node_trace["x"] += (x,)
    node_trace["y"] += (y,)
    node_trace["text"] += (node,)

fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title="Entity Relationship Network",
        showlegend=False,
        hovermode="closest",
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    ),
)

fig.show()

print(f"\n🔍 Entity Analysis:")
print(f"Unique Entities: {len(entities)}")
print(f"Relationships: {len(relationships)}")

## 🎯 IoC Analysis

Let's analyze Indicators of Compromise (IoCs) that have matched against events in our environment. We'll look at:
* IoC types and their distribution
* Match counts and trends
* Sources and categories
* Associated threat actors

In [ ]:
print("Fetching IoC matches...")
iocs = chronicle.list_iocs(
    start_time=start_time,
    end_time=end_time,
    max_matches=1000,
    add_mandiant_attributes=True,
)

# Convert to DataFrame for analysis
ioc_data = []
for match in iocs.get("matches", []):
    # Get the IoC type and value
    ioc_type = next(iter(match.get("artifactIndicator", {}).keys()), "Unknown")
    ioc_value = next(iter(match.get("artifactIndicator", {}).values()), "Unknown")

    ioc_data.append(
        {
            "type": ioc_type,
            "value": ioc_value,
            "sources": ", ".join(match.get("sources", [])),
            "categories": ", ".join(match.get("categories", [])),
            "first_seen": pd.to_datetime(match.get("firstSeenTime")),
            "last_seen": pd.to_datetime(match.get("lastSeenTime")),
            "confidence": match.get("confidenceScore", 0),
            "severity": match.get("severityScore", 0),
        }
    )

df_iocs = pd.DataFrame(ioc_data)

if not df_iocs.empty:
    # Create visualizations
    fig1 = px.pie(df_iocs, names="type", title="IoC Type Distribution")
    fig1.show()

    # Confidence vs Severity scatter plot
    fig2 = px.scatter(
        df_iocs,
        x="confidence",
        y="severity",
        color="type",
        hover_data=["value"],
        title="IoC Confidence vs Severity",
    )
    fig2.show()

    # Timeline of IoC matches
    fig3 = px.histogram(
        df_iocs, x="first_seen", color="type", title="IoC First Seen Timeline"
    )
    fig3.show()

    # Print summary statistics
    print("\n🎯 IoC Summary:")
    print(f"Total IoCs: {len(df_iocs)}")
    print("\nTop IoC Types:")
    print(df_iocs["type"].value_counts())
    print("\nTop Categories:")
    print(df_iocs["categories"].value_counts().head())

    # High severity IoCs
    high_sev = df_iocs[df_iocs["severity"] > 7]
    if not high_sev.empty:
        print("\n⚠️ High Severity IoCs:")
        for _, ioc in high_sev.iterrows():
            print(f"\nType: {ioc['type']}")
            print(f"Value: {ioc['value']}")
            print(f"Severity: {ioc['severity']}")
            print(f"Sources: {ioc['sources']}")
else:
    print("No IoC matches found in the specified time range")

## 🎯 Summary

In this notebook, we've demonstrated the power of the Chronicle Security Operations SDK by:
* Analyzing alert patterns and severity distributions
* Investigating cases and their relationships to alerts
* Visualizing entity relationships and network connections

The SDK provides a powerful interface for security analysis and investigation, making it easy to:
* Query security events
* Track alerts and cases
* Investigate entities
* Visualize security data

For more information, visit the [Chronicle Security Operations documentation](https://cloud.google.com/chronicle/docs).